In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run LandTrendr over Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook uses Landsat and Sentinel 2 composites from the previous notebook inside the LandTrendr temporal setmentation algorithm

Learn more about [LT-GEE-Guide](https://emapr.github.io/LT-GEE/).

### Objective

In this tutorial, you learn how to create and manipulate LandTrendr outputs.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Creating LandTrendr outputs
- Manipulating EE array image objects to get meaningful data out of LandTrendr outputs

In [12]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [2]:
workbench_url = 'https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [11]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

# aml.batchCopy('projects/rcr-gee/assets/landTrendr-lcms-training-module-2',export_landTrendr_collection,outType = 'imageCollection')
# aml.batchDelete('projects/rcr-gee/assets/landTrendr-lcms-training-module-2')
# print('Done')

Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr
['projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NBR_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NDVI_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_brightness_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_greenness_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_nir_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_red_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_swir1_yrs1983-2023_jds152-151', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_swir2_yrs1983-2023_jds152-151', 'projects/rcr-gee/ass

In [8]:
Map.clearMap()
Map.proxy_url = workbench_url
# Index to use to demonstrate change detection capabilities of LandTrendr
#Choose band or index
#NBR, NDMI, and NDVI tend to work best
#Other good options are wetness and tcAngleBG
# Must include 'red','nir','swir1' in order to visualize as false color composites in next code block
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Define landtrendr params
run_params = { \
  'maxSegments':            6,\
  'spikeThreshold':         0.9,\
  'vertexCountOvershoot':   3,\
  'preventOneYearRecovery': False,\
  'recoveryThreshold':      0.25,\
  'pvalThreshold':          0.05,\
  'bestModelProportion':    0.75,\
  'minObservationsNeeded':  6\
}



# Bring in composites and pull info from them
composites = ee.ImageCollection(export_composite_collection)
print(composites.size().getInfo())
props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']


proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']
    
transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('done')

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']
34
1983 2023
done


In [7]:
# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [19]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

#Run LANDTRENDR
for bandName in bandNames:

  # Select the band and run LandTrendr
  run_params['timeSeries'] = composites.select([bandName])
  rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

  Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

  # Notice the raw LandTrendr output is in GEE's image array format
  # We'll need to manipulate the raw output a bit to save on storage space
  # Mask out non vertex values to use less storage space
  ltArray = rawLT.select(['LandTrendr'])
  rmse = rawLT.select(['rmse'])
  vertices = ltArray.arraySlice(0,3,4)
  ltArray = ltArray.arrayMask(vertices)

  # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
  ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
  rawLTForExport=ltArray.addBands(rmse)
  Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

  # Show how the compressed vertex-only values can be decompressed later
  decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
  Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

  # Join the raw and fited values
  fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
  ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
  Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

  # Export LT array image
  # Set some properties that will be uses later
  rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
  rawLTForExport =rawLTForExport.set(run_params)
  exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
  exportPath = exportPathRoot + '/'+ exportName
  # Export output
  getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Raw red
Adding layer: LT Vertex Values Only red
Adding layer: Decompressed LT Output red
Adding layer: Raw and LT Fitted red
pyramiding object: {'.default': 'sample'}
Asset currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw nir
Adding layer: LT Vertex Values Only nir
Adding layer: Decompressed LT Output nir
Adding layer: Raw and LT Fitted nir
pyramiding object: {'.default': 'sample'}
Asset currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw swir1
Adding layer: LT Vertex Values Only swir1
Adding layer: Decompressed LT Output swir1
Adding layer: Raw and LT Fitted swir1
pyramiding object: {'.default': 'sample'}
Asset currently exists or is being exported and overwrite = False. Export not st

In [14]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(exportPathRoot, type = 'imageCollection')

print('done')

['projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NBR_yrs2010-2022_jds273-272', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NDVI_yrs2010-2022_jds273-272', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_nir_yrs2010-2022_jds273-272', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_red_yrs2010-2022_jds273-272', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_swir1_yrs2010-2022_jds273-272', 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_swir2_yrs2010-2022_jds273-272']
Deleting: projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NBR_yrs2010-2022_jds273-272
Deleting: projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_NDVI_yrs2010-2022_jds273-272
Deleting: projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_nir_yrs2010-2022_jds273-272
Deleting: projects/rcr-gee/assets/landTrendr-lcms-training-module-2/LT_Raw_red_yrs2010-2022_jds273-272
Delet

In [16]:
Map.clearMap()

# View exported LT output
lt = ee.ImageCollection(exportPathRoot)
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
lt_fit = changeDetectionLib.batchSimpleLTFit(lt,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection for charting (opacity set to 0 so it will chart but not be visible)
Map.addLayer(lt_fit,{'opacity':0},'LT Fit TS')

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')

Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Synthetic Composite Timelapse')


# Join the raw and fited values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))
print(ltJoined.first().bandNames().getInfo())
Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Fit TS


AttributeError: 'NoneType' object has no attribute 'serialize'